Pontificia Universidad Católica de Chile <br>
Departamento de Ciencia de la Computación <br>
IIC2433 - Minería de Datos
<br>

<center>
    <h2> Tarea 4 </h2>
    <h1> Clustering, Asociaciones y Redes Bayesianas </h1>
    <p>
        Profesor Marcelo Mendoza<br>
        Primer Semestre 2025<br>    
        Fecha de entrega: 23 de mayo
    </p>
    <br>
</center>

<br>

---

## Indicaciones

Deberás entregar **SOLO** el archivo .ipynb en el buzón respectivo en canvas.

**IMPORTANTE**:
- Se te dará puntaje tanto por código como por la manera en la que respondas las preguntas planteadas. Es decir, si tienes un código perfecto pero este no es explicado o no se responden preguntas asociadas a este, no se tendrá el puntaje completo.
- El notebook debe tener todas las celdas de código ejecutadas. Cualquier notebook que no las tenga no podrá ser corregido.
- El carácter de esta tarea es **INDIVIDUAL**. Cualquier instancia de copia resultará en un 1,1 como nota de curso.

---

## Librerías

A continuación se encuentran las librerías necesarias para elaborar la tarea. Recuerda ejecutar la celda antes de comenzar.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans, DBSCAN
from sklearn.mixture import GaussianMixture
from sklearn.neighbors import NearestNeighbors
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.metrics import silhouette_score
import umap.umap_ as UMAP

from pgmpy.inference import VariableElimination
from pgmpy.models import DiscreteBayesianNetwork
from pgmpy.estimators import BayesianEstimator
from pgmpy.estimators import HillClimbSearch, BDs

from mlxtend.frequent_patterns import apriori, association_rules
from mlxtend.preprocessing import TransactionEncoder

## Contexto

Un empresario multimillonario acaba de comprar un club de fútbol de las 5 mejores ligas del mundo y te ha contactado a ti como experto en Mineria de Datos para que lo ayudes a saber como llevar a su nuevo equipo a los primeros lugares. Para conseguir lo anterior te ha entregado datos de la temporada 2015-2016 de la Bundelisga (Alemania), La Liga (España), Ligue 1 (Francia), Premier League (Inglaterra) y Serie A (Italia) con las estadísticas de los equipos que participaron en esa temporada en sus respectivas ligas para poder encontrar estilos de juegos que le puedan servir para replicar en su nuevo equipo.

## 1. Carga y Preprocesamiento de Datos (5 Pts.)

### 1.1 Cargar Datos

Cargue el dataset `teams.csv`. Indique la cantidad de filas y columnas del conjunto de datos. ¿Existen datos nulos? De ser así, mencione el número de casos por variable y luego elimine todas filas que tengan a lo menos un valor nulo.

In [2]:
df_teams = pd.read_csv("teams.csv")
df_teams = df_teams.drop(["Unnamed: 0"], axis = 1)

In [3]:
df_teams.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 78 entries, 0 to 77
Data columns (total 27 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   team                         78 non-null     object 
 1   total_shots                  78 non-null     int64  
 2   onTarget                     78 non-null     int64  
 3   goals                        78 non-null     int64  
 4   xG_total                     78 non-null     float64
 5   Shot_to_goal_ratio           78 non-null     float64
 6   Shot_onTarget_to_goal_ratio  78 non-null     float64
 7   xG/shot                      78 non-null     float64
 8   recoveries                   78 non-null     int64  
 9   recoveries_f3                78 non-null     int64  
 10  duels_won                    78 non-null     int64  
 11  fouls_committed              78 non-null     int64  
 12  shots_against                78 non-null     int64  
 13  shots_onTarget_against

El dataset contiene 77 filas (Sin contar el header) y 27 columnas 

Además aparentemente (Al menos según pandas y posterior indagación dentro del archivo .csv) no existen datos nulos

### 1.2 Descripción del dataset (1 Pts.)

Describa brevemente cuantas columnas tiene el dataset y defina que significa cada columna del dataset. Para lo anterior te puedes apoyar de las siguientes páginas webs:

- [Definición Métricas Opta](https://theanalyst.com/2024/07/opta-football-stats-definitions)
- [Explicación xG Opta](https://theanalyst.com/2023/08/what-is-expected-goals-xg)
- [Definición Métricas StatsPerform](https://www.statsperform.com/opta-event-definitions/)
- [Glosario WhoScored](https://www.whoscored.com/glossary)

In [7]:
df_teams

,team,total_shots,onTarget,goals,xG_total,Shot_to_goal_ratio,Shot_onTarget_to_goal_ratio,xG/shot,recoveries,recoveries_f3,...,ppda,total_passes,f3_passes,short_passes,long_passes,average_pass_length,crosses,ranking,points_per_match,groups
0,Real Madrid,717,287,108,80.794913,15.062762,37.630662,0.112685,1715,448,...,12.785114,23279,2225,15556,2388,18.465597,683,2,2.37,1
1,Napoli,664,244,78,64.630668,11.746988,31.967213,0.097335,1847,452,...,13.472731,26906,2671,18602,2099,16.582880,519,2,2.16,1
2,Tottenham Hotspur,656,249,68,62.356203,10.365854,27.309237,0.095055,1976,504,...,10.571661,20624,2527,11906,2577,20.121036,431,3,1.84,1
3,Liverpool,635,203,62,58.373657,9.763780,30.541872,0.091927,1999,475,...,10.999069,21464,2427,12984,2298,18.959322,511,8,1.58,2
4,Bayern Munich,622,258,78,73.353521,12.540193,30.232558,0.117932,1604,429,...,11.116863,25862,2633,18128,2090,18.200877,561,1,2.59,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73,Hamburger SV,375,130,37,37.730126,9.866667,28.461538,0.100614,1690,314,...,14.357576,15534,1992,7753,2492,20.968141,316,10,1.21,2
74,Eintracht Frankfurt,364,128,32,33.644640,8.791209,25.000000,0.092430,1679,357,...,14.506135,15431,1915,8060,2410,20.244928,425,16,1.06,4
75,Darmstadt 98,347,114,38,36.581159,10.951009,33.333333,0.105421,1489,383,...,14.103314,9549,2207,2770,2634,24.371304,338,14,1.12,3
76,Villarreal,347,129,42,37.071917,12.103746,32.558140,0.106835,1726,333,...,14.630580,17546,2038,9808,2264,19.881607,342,4,1.68,1


In [5]:
len(df_teams["team"].unique())

78

Las columnas del dataset y lo que representa cada una es:
1) team: Representa el equipo, veo que este tiene un valor unico por columna
2) total_shots: Representa el número de veces que un equipo ha hecho un tiro
3) onTarget: según la definición de métricas Opta, esta columna representaría el número de veces que se ha pateado la pelota hacia portería, incluyendo las veces que se ha hecho un gol (o score) como aquellas en las que el arquero lo ha evitado
4) goals: Número de veces que un equipo ha hecho un gol
5) xG_total: si xG representa la probabilidad de que un tiro resulte en gol, xG_total representa la suma de todos los xG de los tiros de un equipo
6) Shot_to_goal_ratio: Sería la división entre el número de goles partido por el número total_shots de un equipo
7) Shot_onTarget_to_goal_ratio: Sería la división entre el número de goles partido por el número onTarget de un equipo
8) xG/shot: Corresponde a la fracción de la probabilidad de que un tiro resulte en gol partido por el número de tiros
9) recoveries: Número de veces en la que un equipo ha recuperado la pelota en situaciones en las que ninguno de los equipos la tiene o cuando la pelota ha sido jugada directamente hacia el por un equipo rival, así asegurando la posesión de la pelota para el equipo
10) recoveries_f3: Es lo mismo que recoveries pero hecha en la zona F3 del campo de futbol, siendo F3 la zona defensiva del equipo rival (el tercio final del campo contrario)
11) duels_won: Esta métrica aplicada a un equipo representaría el número de duelos individuales entre jugadores del equipo contra jugadores de equipos rivales que los jugadores del equipo han ganado. Siendo un duelo una disputa directa por el balón
12) fouls_committed: Número de veces que jugadores del equipo han infringido reglas del juego.
13) shots_against: Número de intentos de gol que el equipo rival ha realizado
14) shots_onTarget_against: Lo mismo que shots_against solo que estos se refieren a tiros que iban dirigidos directos a portería y cuya línea de defensa es solo el arquero
15) xGA: Es lo mismo que xG pero tomando solo los tiros hechos por el equipo rival
16) goals_against: Es el número de goles que les han hecho al equipo3
17)          

### 1.3 Normalización (1 Pts.)

Realice el proceso de normalización sobre la matriz de variables explicativas utilizando el método StandardScaler. A partir de aquí, tiene que usar esta matriz para los experimentos. Para lo anterior, elimine las columnas `team`, `ranking`, `points_per_match` y `groups`, pero mantenga una copia del dataframe original, ya que lo necesitaras en secciones posteriores.

### 1.4 Matriz de correlación (2 Pts.)

A partir del dataframe normalizado en la sección anterior construya la matriz de correlación. ¿Que variables tienen alta correlación ($> 0.8$)?

## 2. Clustering (15 Pts.)

### 2.1 K-Means (5 Pts.)

#### 2.1.1 Explicación teórica (2 Pts.)

Investigue sobre `K-means` y entregue una explicación de como funciona. `No es necesario una explicación matemática`. Se espera que tu explicación contenga la respuesta a las siguientes preguntas:
1. ¿Qué tipo de algoritmo es?
2. ¿Para qué sirve?
3. ¿Cómo funciona?
4. ¿Qué es un centroide?
5. ¿Que es la inercia en este contexto?

**❗IMPORTANTE❗** No utilices ChatGPT u otro modelo de lenguaje para responder esta pregunta. Será evaluado que cites al menos 1 recurso bibliográfico que hayas utilizado. Cualquier información que no tenga una referencia clara no se le dará puntaje.

#### 2.1.2 Número de clusters (2 Pts.)

La libreria [scikit-learn](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html) tiene implementado este algoritmo. Grafique los valores de *inertia* y silhoutte score para un rango de 2 a 10 clusters. Determine el número óptimo de clusters.

#### 2.1.3 Ejecutar K-Means (1 Pts.)

**Ejecute K-Means con un número de clusters igual a 4.** Una vez obtenidos los clusters imprima las posiciones en que terminaron esos equipos:

- ¿Existe alguna relación entre los clusters obtenidos y la posición que obtuvieron los equipos de ese cluster respectivo?
- Describa cada cluster según la posición de los clubes dentro de los clusters y otras estadísticas presentes en el dataset.

Finalmente, calcule *silhouette score* con los valores escogidos.

### 2.2 DBSCAN (5 Pts.)

Usando *k-dist plot* para MinPts=3, encuentre que rango de EPS deberiamos evaluar. Una vez obtenidos los clusters realice el mismo análisis de la sección anterior:

- ¿Existe alguna relación entre los clusters obtenidos y la posición que obtuvieron los equipos de ese cluster respectivo?
- Describa cada cluster según la posición de los clubes dentro de los clusters y otras estadísticas presentes en el dataset.

Finalmente, calcule *silhouette score* con los valores escogidos

### 2.3 GMM (5 Pts.)

#### 2.3.1 Explicación AIC (2 Pts.)

Investigue sobre las métricas `AIC`, y conteste las siguientes preguntas:

- ¿Qué es la métrica `AIC`?
- ¿Qué penaliza esta métrica?
- ¿Qué casos de usos tiene esta métrica?
- Para el caso de GMM, ¿se busca maximizar o minimizar esta métrica? ¿Por qué?

**❗IMPORTANTE❗** No utilices ChatGPT u otro modelo de lenguaje para responder esta pregunta. Será evaluado que cites al menos 1 recurso bibliográfico que hayas utilizado. Cualquier información que no tenga una referencia clara no se le dará puntaje.

#### 2.3.2 Obtener AIC (1 Pts.)

Determine el número óptimo de componentes para el modelo Gaussian Mixture Model utilizando la métrica AIC. Considere un rango de número de componentes entre 1 y 40.

Luego, grafique la curva entre número de componentes y la métrica AIC

#### 2.3.3 Implementación GMM (2 Pts.)

Escoja el número de componentes que minimiza AIC para entrenar GMM. Una vez obtenidos los clusters realice el mismo análisis de la sección anterior:

- ¿Existe alguna relación entre los clusters obtenidos y la posición que obtuvieron los equipos de ese cluster respectivo?
- Describa cada cluster según la posición de los clubes dentro de los clusters y otras estadísticas presentes en el dataset.

Finalmente, calcule *silhouette score* con los valores escogidos.

## 3. Visualización (10 Pts.)

En esta sección se espera que grafiques los datos coloreados según su cluster utilizando PCA, t-SNE y UMAP.

### 3.1 PCA n=2 (3 Pts.)

Realiza una gráfica de PCA con 2 componentes coloreando cada dato según su cluster para los resultados obtenidos con K-Means, DBSCAN y GMM.

### 3.2 t-SNE (2 Pts)

Realiza una gráfica de t-SNE con 2 componentes coloreando cada dato según su cluster para los resultados obtenidos con K-Means, DBSCAN y GMM.

### 3.3 UMAP (3 Pts.)

Realiza una gráfica de UMAP con 2 componentes coloreando cada dato según su cluster para los resultados obtenidos con K-Means, DBSCAN y GMM.

### 3.4 Conclusiones (2 Pts.)

En base a lo observado en los gráficos obtenidos en la sección anterior conteste:

- ¿Que método de reducción de dimensionalidad permite ver de mejor manera la agrupación de equipos?
- ¿Observaste alguna relación entre los silhoutte score obtenidos y como se visualizaron los clusters en los gráficos anteriores? ¿Qué método de clusterización se comportó mejor al graficarlo?
- ¿Existió algún cluster que se mantuvo entre las distintas técnicas de clusterización?

## 4. Redes Bayesianas (15 Pts.)

A continuación debes realizar un análisis de redes bayesianas sobre el dataset, con la finalidad de poder inferir la probabilidad de que un equipo pertenzca a cierto grupo o no, dadas las características que se presentan en el dataset.

### 4.1 Explicación Teórica (5 Pts.)

Investiga sobre `Redes Bayesianas` y entrega una explicación de cómo funciona. `No es necesaria una explicación matemática`, el objetivo es que puedas tomar lo que aprendiste en clases o buscando en internet, y logres exponerlo de manera sintetizada para demostrar tu aprendizaje. Se espera que tu explicación contenga la respuesta a las siguientes preguntas:
1. ¿Qué es?
2. ¿Para qué sirve?
3. ¿Cómo se construye?
4. ¿Qué es un nodo y una arista en una red bayesiana?
5. ¿Qué es un CPD?

**❗IMPORTANTE❗** No utilices ChatGPT u otro modelo de lenguaje para responder esta pregunta. Será evaluado que cites al menos 1 recurso bibliográfico que hayas utilizado. Cualquier información que no tenga una referencia clara no se le dará puntaje.

### 4.2 Aprender la estructura de la red bayesiana (3 Pts.)

Del DataFrame original, elimine la columna `team`.
Luego, utilice la librería pgmpy en Python para aplicar el algoritmo Hill Climb Search con el criterio de puntuación BDs y encontrar la estructura óptima de una red bayesiana. Puedes guiarte de las actividades vistas en clases.

(Sugerencia: la clase `BDs` se importa de forma simiar a `BIC`, es decir, `from pgmpy.estimators import BDs`)

### 4.3 Encontrar la estructura óptima (2 Pts.)

En esta sección, debes entrenar la red bayesiana con los datos de entrenamiento usando la estructura óptima encontrada en la sección anterior, ajustando los parámetros usando priors de Dirichlet con cuentas uniformes. Luego, debes imprimir las probabilidades condicionales de la red bayesiana.

### 4.4 Inferencia sobre los datos (5 Pts.)

Finalmente, debes realizar inferencia sobre la red bayesiana. Para esto, se te pide que calcules la probabilidad de que un equipo pertenezca al grupo `groups` dadas las siguientes características

- Equipo 1: 'fouls_committed': 706, 'duels_won': 433
- Equipo 2: 'total_shots': 439, 'total_passes': 15919
- Equipo 3: 'goals': 80, 'goals_against': 51, 'onTarget': 137

¿Que limitaciones cree que tiene este modelo para el problema?

## 5. Asociaciones (15 Pts.)

Debido a tu gran desempeño en tu labor anterior detectando los distintos tipos de equipos en las 5 grandes ligas de fútbol, el empresario multimillonario te quiere encargar una nueva misión. Se ha enterado que el último campéon de la Eurocopa ha sido España, por lo que quiere fichar jugadores de esta selección que se relacionen bien entre ellos para poder ficharlos. Para conseguir lo anterior, te ha entregado un dataset con las secuencias de pases de los jugadores de la Selección Española durante sus partidos en la Eurocopa 2024 y tu labor será encontrar los jugadores que mejor se asocian.

Una secuencia de pases se entiende como todos los jugadores que se dieron pases entre ellos antes de que el equipo haya perdido el balón o el juego se haya detenido por algún motivo. El dataset contiene lineas con los nombres de los jugadores que participaron en una secuencia de pases separados por ;.

### 5.1 Cargar Datos (2 Pts.)

Carga el dataset `passes.csv` y dejando los datos como una lista de listas.

### 5.2 Mostrar frecuencia (3 Pts.)

A continuación debes mostrar la frecuencia de participación de los jugadores en las secuencias de pases ordenadas de manera descendente.

### 5.3 Transformar datos (3 Pts)

Utiliza `TransactionEncoder()` para codificar las secuencias de pases en vectores.

### 5.4 Matriz de co-ocurrencia (2 Pts.)

Genera la matriz de co-ocurrencia para observar con qué frecuencia los jugadores participan entre sí en las secuencias de pases. En base a la matriz contesta las siguientes preguntas:

- ¿Qué jugadores se observa que estan relacionados?
- Revisando las formaciones de España durante la Eurocopa, ¿se observa alguna relación entre las posiciones en el campo y los jugadores con mayor relación entre ellos?

[España v/s Croacia](https://www.sofascore.com/football/match/croatia-spain/YTbspUb#id:11873907)

[España v/s Italia](https://www.sofascore.com/football/match/italy-spain/YTbshUb#id:11873901)

[España v/s Albania](https://www.sofascore.com/football/match/spain-albania/PTbsYTb#id:11873904)

[España v/s Georgia](https://www.sofascore.com/football/match/georgia-spain/YTbsnVb#id:11874031)

[España v/s Alemania](https://www.sofascore.com/football/match/germany-spain/YTbslUb#id:11874027)

[España v/s Francia](https://www.sofascore.com/football/match/germany-spain/YTbslUb#id:11874027)

[España v/s Inglaterra](https://www.sofascore.com/football/match/england-spain/YTbsnUb#id:11874023)

### 5.5 Algoritmo a-priori (5 Pts.)

Aplica el algoritmo Apriori con min_support=0.1 y analiza los resultados obtenidos.

A continuación, genera las reglas de asociación utilizando association_rules, con un umbral min_threshold=0.3, y en base a los resultados obtenidos conteste las siguientes preguntas:

- ¿Qué par de jugadores recomendaría que se ficharan juntos?
- ¿Crees que este método es suficiente para recomendar fichaje de jugadores? ¿Por qué?